In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from imblearn.combine import SMOTEENN

In [2]:
train_feat = pd.read_json("./features_preprocess/features_train_hist.json")
test_feat = pd.read_json("./features_preprocess/features_test_hist.json")
val_feat = pd.read_json("./features_preprocess/features_validation_hist.json")

train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")

In [3]:
train_feat.loc[:, "image_id"] = train_feat["filepath"].map(lambda path :  int(path.split("/")[-1].rstrip(".png")))
val_feat.loc[:, "image_id"] = val_feat["filepath"].map(lambda path :  int(path.split("/")[-1].rstrip(".png")))
test_feat.loc[:, "image_id"] = test_feat["filepath"].map(lambda path :  int(path.split("/")[-1].rstrip(".png")))

In [4]:
val_dataset = train_dataset[train_dataset.image_id.isin(val_feat.image_id)]
train_dataset_actual = train_dataset[train_dataset.image_id.isin(train_feat.image_id)]

In [5]:
test_dataset = test_dataset.merge(test_feat, on="image_id")
val_dataset = val_dataset.merge(val_feat, on="image_id")
train_dataset_actual = train_dataset_actual.merge(train_feat, on="image_id")


In [6]:
relevant_columns = ["age", "features"]
stepval = train_dataset_actual.loc[pd.notna(train_dataset_actual.age), relevant_columns].to_numpy()
train_smoteIn = np.array([np.append(np.array(elem[0]), (np.array(elem[1])))  for elem in stepval])
label = train_dataset_actual.loc[pd.notna(train_dataset_actual.age), "cancer"].to_numpy()

In [7]:
smote_enn = SMOTEENN(random_state=0, sampling_strategy="all")
train_x_resampled, train_y_resampled = smote_enn.fit_resample(train_smoteIn, label)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_x_resampled)
scaled = scaler.transform(train_x_resampled)

In [10]:
model = RandomForestClassifier(max_depth=15, random_state=0)
model.fit(scaled, train_y_resampled)

RandomForestClassifier(max_depth=15, random_state=0)

In [11]:
relevant_columns = ["age", "features"]
stepval = val_dataset.loc[pd.notna(val_dataset.age), relevant_columns].to_numpy()
val_in = np.array([np.append(np.array(elem[0]), (np.array(elem[1])))  for elem in stepval])
label = val_dataset.loc[pd.notna(val_dataset.age), "cancer"].to_numpy()

In [12]:
scaled_val = scaler.transform(val_in)

In [13]:
val_predict = model.predict(scaled_val)

In [14]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(label, val_predict)

0.508596408552312

In [15]:
relevant_columns = ["age", "features"]
stepval = test_dataset.loc[pd.notna(test_dataset.age), relevant_columns].to_numpy()
test_in = np.array([np.append(np.array(elem[0]), (np.array(elem[1])))  for elem in stepval])
label = test_dataset.loc[pd.notna(test_dataset.age), "cancer"].to_numpy()

In [16]:
scaled_val = scaler.transform(test_in)


In [17]:
test_predict = model.predict(scaled_val)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(label, test_predict)

0.5381999318273373